In [4]:
# pydantic==1.10.9

In [1]:
import langchain
import pandas

In [2]:
from langchain.chains import RetreiveChain

ImportError: cannot import name 'RetreiveChain' from 'langchain.chains' (/Users/samuelbamgbola/Documents/Gospel-Companion/lib/python3.7/site-packages/langchain/chains/__init__.py)

In [4]:
# pip install langchain[all]

zsh:1: no matches found: langchain[all]
Note: you may need to restart the kernel to use updated packages.


In [6]:
# pip install langchain --upgrade

Requirement already up-to-date: langchain in /Users/samuelbamgbola/Documents/Gospel-Companion/lib/python3.7/site-packages (0.0.27)
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import llama 2 from langchain
llama = langchain.llms

In [ ]:
# Create a prompt template using langchain for a query of sunday school lesson teacher
# The prompt is a template for a query of sunday school lesson teacher

def answer(query):
    """
    Function housing prompt template
    """

    # Create a prompt template using langchain for a query of sunday school lesson teacher
    # The prompt is a template for a query of sunday school lesson teacher
    prompt = langchain.PromptTemplate()
        # T = teacher, L = lesson, S = scripture, 

